In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
from datetime import datetime
from google.cloud.logging import DESCENDING, Client
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18

In [2]:
# read getPayloads from file.
gp = pd.read_csv('gpFull.txt', delimiter=',')
gp['inserted_at'] = pd.to_datetime(gp['inserted_at'], format="%Y-%m-%d %H:%M:%S.%f")
gp.head()

,slot,ip,ua,req_time,block_hash,proposer_pubkey,inserted_at,slot_boundary
0,6200251,13.51.48.23,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35.808494+00:00,0xa9919dcc29eabc8e0c967108e306dfc300d2425c7912...,0x8a13257bd79486d55e39b5edba59be7db1539789b71b...,2023-04-11 15:30:37.358157,2023-04-11 15:30:35.000000
1,6200260,34.249.7.41,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23.554700+00:00,0x15116fbb7f527090d7f5776926c945b8123224ecb7d4...,0x8763a8e8f8c968cc05f782f8d90e452a1af789724081...,2023-04-11 15:32:25.193994,2023-04-11 15:32:23.000000
2,6200263,167.172.105.217,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59.539492+00:00,0xf4bc26ed3cd6ce65fff3e7c249e0d8b3158e1d75c4eb...,0xb0e4825e9b0f07e407c6521d691464a03ec41352edbc...,2023-04-11 15:33:01.098459,2023-04-11 15:32:59.000000
3,6200264,142.132.159.189,mev-boost/v1.5.0,2023-04-11 15:33:11.715400+00:00,0x96a06b84bb0122783f6dc0f671ffa6c93c3b6acd6a68...,0xb2787d518339828927356da0445d0cde6cdcd3255bc1...,2023-04-11 15:33:13.160390,2023-04-11 15:33:11.000000
4,6200265,52.18.21.3,mev-boost/v1.5.0,2023-04-11 15:33:23.510303+00:00,0xdfbbca4c0b929ea48df4a736ddde6052bd4c2546c841...,0x8a446df54f9b40802ee85eb60c528c7544c8fea24292...,2023-04-11 15:33:25.186221,2023-04-11 15:33:23.000000


In [5]:
gh = pd.read_csv('ghFull.txt', delimiter=',')
gh['reqTime'] = pd.to_datetime(gh['reqTime'], format="%Y-%m-%d %H:%M:%S.%f")
gh.head()

,slot,ip,ua,reqTime
0,6200251,13.51.48.23,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35.422391+00:00
1,6200260,34.249.7.41,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23.365511+00:00
2,6200263,167.172.105.217,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59.152492+00:00
3,6200265,52.18.21.3,mev-boost/v1.5.0,2023-04-11 15:33:23.212169+00:00
4,6200278,52.191.229.43,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:35:59.144297+00:00


In [7]:
len(gh.slot), len(gp.slot)

(1241, 2343)

In [8]:
len(set(gh.slot)), len(set(gp.slot))

(1238, 2343)

In [36]:
gh.slot.value_counts()

6200740    2
6201534    2
6203960    2
6203391    1
6202062    1
          ..
6200669    1
6202718    1
6204767    1
6200672    1
6202959    1
Name: slot, Length: 1238, dtype: int64

In [28]:
merged = pd.merge(gh, gp, on=['ip', 'slot'], how='inner')
merged.head()

,slot,ip,ua_x,reqTime,ua_y,req_time,block_hash,proposer_pubkey,inserted_at,slot_boundary
0,6200251,13.51.48.23,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35.422391+00:00,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35.808494+00:00,0xa9919dcc29eabc8e0c967108e306dfc300d2425c7912...,0x8a13257bd79486d55e39b5edba59be7db1539789b71b...,2023-04-11 15:30:37.358157,2023-04-11 15:30:35.000000
1,6200260,34.249.7.41,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23.365511+00:00,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23.554700+00:00,0x15116fbb7f527090d7f5776926c945b8123224ecb7d4...,0x8763a8e8f8c968cc05f782f8d90e452a1af789724081...,2023-04-11 15:32:25.193994,2023-04-11 15:32:23.000000
2,6200263,167.172.105.217,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59.152492+00:00,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59.539492+00:00,0xf4bc26ed3cd6ce65fff3e7c249e0d8b3158e1d75c4eb...,0xb0e4825e9b0f07e407c6521d691464a03ec41352edbc...,2023-04-11 15:33:01.098459,2023-04-11 15:32:59.000000
3,6200265,52.18.21.3,mev-boost/v1.5.0,2023-04-11 15:33:23.212169+00:00,mev-boost/v1.5.0,2023-04-11 15:33:23.510303+00:00,0xdfbbca4c0b929ea48df4a736ddde6052bd4c2546c841...,0x8a446df54f9b40802ee85eb60c528c7544c8fea24292...,2023-04-11 15:33:25.186221,2023-04-11 15:33:23.000000
4,6200278,52.191.229.43,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:35:59.144297+00:00,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:35:59.400544+00:00,0xef2e8b0784bc9e9077f507e4d6bbfb01473042dd2fbc...,0x99344ff71e0dfb03779af88f71e6ceef719e73fbc8b4...,2023-04-11 15:36:01.095292,2023-04-11 15:35:59.000000


In [29]:
merged = merged.rename(columns={'reqTime': 'header_req', 
                                'req_time': 'payload_req',
                                'ua_x': 'header_ua',  
                                'ua_y': 'payload_ua',
                       })
merged = merged[['slot', 'ip', 'header_ua', 'payload_ua', 'slot_boundary', 'header_req', 'payload_req', 'block_hash', 'proposer_pubkey']]

for col in ['slot_boundary', 'header_req', 'payload_req']:
    merged[col] = pd.to_datetime(merged[col]).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    merged[col] = pd.to_datetime(merged[col])
merged.head()

,slot,ip,header_ua,payload_ua,slot_boundary,header_req,payload_req,block_hash,proposer_pubkey
0,6200251,13.51.48.23,mev-boost/v1.5.0 Go-http-client/1.1,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35,2023-04-11 15:30:35.422391,2023-04-11 15:30:35.808494,0xa9919dcc29eabc8e0c967108e306dfc300d2425c7912...,0x8a13257bd79486d55e39b5edba59be7db1539789b71b...
1,6200260,34.249.7.41,mev-boost/v1.5.0 Go-http-client/1.1,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23,2023-04-11 15:32:23.365511,2023-04-11 15:32:23.554700,0x15116fbb7f527090d7f5776926c945b8123224ecb7d4...,0x8763a8e8f8c968cc05f782f8d90e452a1af789724081...
2,6200263,167.172.105.217,mev-boost/v1.5.0 okhttp/4.10.0,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59,2023-04-11 15:32:59.152492,2023-04-11 15:32:59.539492,0xf4bc26ed3cd6ce65fff3e7c249e0d8b3158e1d75c4eb...,0xb0e4825e9b0f07e407c6521d691464a03ec41352edbc...
3,6200265,52.18.21.3,mev-boost/v1.5.0,mev-boost/v1.5.0,2023-04-11 15:33:23,2023-04-11 15:33:23.212169,2023-04-11 15:33:23.510303,0xdfbbca4c0b929ea48df4a736ddde6052bd4c2546c841...,0x8a446df54f9b40802ee85eb60c528c7544c8fea24292...
4,6200278,52.191.229.43,mev-boost/v1.5.0 okhttp/4.10.0,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:35:59,2023-04-11 15:35:59.144297,2023-04-11 15:35:59.400544,0xef2e8b0784bc9e9077f507e4d6bbfb01473042dd2fbc...,0x99344ff71e0dfb03779af88f71e6ceef719e73fbc8b4...


In [33]:
noip = merged.drop(labels=['ip'], axis=1)
noip.head()

,slot,header_ua,payload_ua,slot_boundary,header_req,payload_req,block_hash,proposer_pubkey
0,6200251,mev-boost/v1.5.0 Go-http-client/1.1,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:30:35,2023-04-11 15:30:35.422391,2023-04-11 15:30:35.808494,0xa9919dcc29eabc8e0c967108e306dfc300d2425c7912...,0x8a13257bd79486d55e39b5edba59be7db1539789b71b...
1,6200260,mev-boost/v1.5.0 Go-http-client/1.1,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-11 15:32:23,2023-04-11 15:32:23.365511,2023-04-11 15:32:23.554700,0x15116fbb7f527090d7f5776926c945b8123224ecb7d4...,0x8763a8e8f8c968cc05f782f8d90e452a1af789724081...
2,6200263,mev-boost/v1.5.0 okhttp/4.10.0,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:32:59,2023-04-11 15:32:59.152492,2023-04-11 15:32:59.539492,0xf4bc26ed3cd6ce65fff3e7c249e0d8b3158e1d75c4eb...,0xb0e4825e9b0f07e407c6521d691464a03ec41352edbc...
3,6200265,mev-boost/v1.5.0,mev-boost/v1.5.0,2023-04-11 15:33:23,2023-04-11 15:33:23.212169,2023-04-11 15:33:23.510303,0xdfbbca4c0b929ea48df4a736ddde6052bd4c2546c841...,0x8a446df54f9b40802ee85eb60c528c7544c8fea24292...
4,6200278,mev-boost/v1.5.0 okhttp/4.10.0,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-11 15:35:59,2023-04-11 15:35:59.144297,2023-04-11 15:35:59.400544,0xef2e8b0784bc9e9077f507e4d6bbfb01473042dd2fbc...,0x99344ff71e0dfb03779af88f71e6ceef719e73fbc8b4...


In [35]:
noip.to_csv('gh-gp.csv', index=False)